In [10]:
dd_image_path = "cfreds_2015_data_leakage_pc.dd"
dd_image_offset = 206848
search_pattern = "gmreadme.txt"

In [11]:
import pytsk3
import argparse

# Define the image and the file pattern to search for
# image_path = '../cfreds_2015_data_leakage_pc.dd'
# search_pattern = 'gmreadme.txt'
def get_entry(image_path, image_offset, search_pattern):
    # Open the disk image
    img = pytsk3.Img_Info(image_path)

    # File system offset (assumed to be provided as in your command)
    fs_offset = image_offset

    # Open the file system
    fs = pytsk3.FS_Info(img, offset=fs_offset*512)

    # Define a recursive function to list files
    def list_files(directory, path='/'):
        for entry in directory:
            # Skip ".", ".." or directory entries without a name
            if not entry.info.name or entry.info.name.name in [b".", b".."]:
                continue
            try:
                file_name = entry.info.name.name.decode('utf-8')
                # print(file_name)
                file_path = f"{path}{file_name}"
                current_path = f"{path}{entry.info.name.name.decode('utf-8')}"

                if entry.info.meta.type == pytsk3.TSK_FS_META_TYPE_DIR:
                    # print(entry.info.name.name.decode('utf-8'))
                    # print(current_path)
                    sub_directory = pytsk3.Directory(fs, current_path)
                    list_files(sub_directory, path=f"{current_path}/")
                else:
                    if search_pattern in file_name:
                        print(f"Found: {entry.info.meta.addr} | {file_path}")
            except Exception:
                continue


    # Start listing from the root directory
    root_dir = fs.open_dir(path="/")
    list_files(root_dir)

if __name__ == "__main__":
    # Print result
    get_entry(dd_image_path, dd_image_offset, search_pattern)


Found: 27331 | /Windows/System32/drivers/gmreadme.txt
Found: 27330 | /Windows/SysWOW64/drivers/gmreadme.txt
Found: 27331 | /Windows/winsxs/amd64_microsoft-windows-audio-mmecore-other_31bf3856ad364e35_6.1.7600.16385_none_e8f2b9ab2a40e84d/gmreadme.txt
Found: 27330 | /Windows/winsxs/x86_microsoft-windows-audio-mmecore-other_31bf3856ad364e35_6.1.7600.16385_none_8cd41e2771e37717/gmreadme.txt
